<a href="https://colab.research.google.com/github/sihuniiii/mytest/blob/main/Minimizer_indexing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import random

def generate_reference_to_file(filename, length, chunk_size=10_000_000):
    bases = ['A', 'C', 'G', 'T']
    with open(filename, 'w') as f:
        written = 0
        while written < length:
            to_write = min(chunk_size, length - written)
            seq_chunk = ''.join(random.choices(bases, k=to_write))
            f.write(seq_chunk)
            written += to_write

def simulate_ancient_reads_to_file(
    reference_file,
    reads_filename,
    truth_filename,
    read_len,
    num_reads,
    mutation_rate=0.02
):
    with open(reference_file, 'r') as f:
        reference_seq = f.read().strip()
    L = len(reference_seq)

    with open(reads_filename, 'w') as rf, open(truth_filename, 'w') as tf:
        for i in range(num_reads):
            start = random.randint(0, L - read_len)
            original = list(reference_seq[start : start + read_len])
            mutated = []
            for j, base in enumerate(original):
                prob = mutation_rate
                if j < 5 or j >= read_len - 5:
                    prob *= 5
                if random.random() < prob:
                    if base == 'C':
                        mutated.append('T')
                    elif base == 'G':
                        mutated.append('A')
                    else:
                        mutated.append(random.choice(['A', 'C', 'G', 'T']))
                else:
                    mutated.append(base)
            read_str = ''.join(mutated)
            rf.write(read_str + '\n')
            tf.write(f"{start}\n")

def run_simulation():
    settings = [
        (1_000_000,100_000,"1M", "100K"),
        (10_000_000,1_000_000,"10M", "1M"),
        (100_000_000,10_000_000,"100M","10M"),
        (1_000_000_000,100_000_000,"1T", "100M"),
    ]

    # Reference 생성
    for ref_len, _, ref_tag, _ in settings:
        ref_filename = f"reference_{ref_tag}.txt"
        if not os.path.exists(ref_filename):
            generate_reference_to_file(ref_filename, ref_len)

    for ref_len, num_reads, ref_tag, read_tag in settings:
        ref_filename   = f"reference_{ref_tag}.txt"
        reads_filename = f"mammoth_reads_{read_tag}.txt"
        truth_filename = f"ground_truth_{read_tag}.txt"
        read_len = 100

        if not (os.path.exists(reads_filename) and os.path.exists(truth_filename)):
            simulate_ancient_reads_to_file(
                reference_file=ref_filename,
                reads_filename=reads_filename,
                truth_filename=truth_filename,
                read_len=read_len,
                num_reads=num_reads,
                mutation_rate=0.01
            )

if __name__ == "__main__":
    run_simulation()

In [ ]:
import os

#파일 불러오기
def load_reference_to_string(filename):
    with open(filename, 'r') as f:
        return f.read().strip()

def load_reads_from_file(filename):
    with open(filename, 'r') as f:
        return [line.strip() for line in f]

def load_truth_positions(filename):
    return [int(line.strip()) for line in open(filename, 'r')]


# Minimizer 인덱스
def build_minimizer_index(reference, k=20, w=8, max_occ=500):
    index = {}
    num_kmers = len(reference) - k + 1
    for i in range(num_kmers - w + 1):
        window_kmers = [reference[j:j + k] for j in range(i, i + w)]
        mn = min(window_kmers)
        mn_pos = i + window_kmers.index(mn)
        index.setdefault(mn, []).append(mn_pos)

    filtered = {m: poses for m, poses in index.items() if len(poses) <= max_occ}
    return filtered

# 매칭 알고리즘
def minimizer_match(
    reference,index,read,
    k=20,w=8,
    max_mismatch=2,seed_min=2
):
    read_kmers = [read[i:i + k] for i in range(len(read) - k + 1)]
    read_min_pairs = []
    for i in range(len(read_kmers) - w + 1):
        window = read_kmers[i:i + w]
        mn = min(window)
        mn_idx = window.index(mn)
        read_pos = i + mn_idx
        read_min_pairs.append((mn, read_pos))

    delta_counts = {}
    for mn, rpos in read_min_pairs:
        if mn not in index:
            continue
        for refpos in index[mn]:
            delta = refpos - rpos
            delta_counts[delta] = delta_counts.get(delta, 0) + 1

    candidates = [d for d, cnt in delta_counts.items() if cnt >= seed_min]
    best_pos, best_mm = -1, max_mismatch + 1

    for delta in candidates:
        if delta < 0 or delta + len(read) > len(reference):
            continue
        window_seq = reference[delta : delta + len(read)]
        mismatches = sum(1 for a, b in zip(read, window_seq) if a != b)
        if mismatches <= max_mismatch and mismatches < best_mm:
            best_mm = mismatches
            best_pos = delta

    return best_pos, best_mm

# 복원 함수
def reconstruct_genome_with_reads(
    reference,
    reads,
    truth_positions,
    index,
    k=20, w=8, max_mismatch=2, seed_min=2
):
    reconstructed = list(reference)
    matched_reads = 0
    total_reads = len(reads)

    for i, read in enumerate(reads):
        true_pos = truth_positions[i]
        pred_pos, mm = minimizer_match(
            reference, index, read,
            k=k, w=w, max_mismatch=max_mismatch, seed_min=seed_min
        )
        # mismatch 허용 범위에서 복원
        if pred_pos == true_pos and mm <= max_mismatch:
            for j, base in enumerate(read):
                reconstructed[pred_pos + j] = base
            matched_reads += 1

    return ''.join(reconstructed), matched_reads, total_reads

# 정확도 계산
def evaluate_reconstruction(reference, reconstructed):
    assert len(reference) == len(reconstructed)
    L = len(reference)
    matches = sum(1 for a, b in zip(reference, reconstructed) if a == b)
    return matches / L

# 매핑 및 평가
def run_mapping_and_evaluation():

    K = 20
    W = 8
    MAX_OCC = 500
    MAX_MM = 2
    SEED_MIN = 2

    pairs = [
        ("reference_1M.txt",  "mammoth_reads_100K.txt",  "ground_truth_100K.txt"),
        ("reference_10M.txt", "mammoth_reads_1M.txt",   "ground_truth_1M.txt"),
        ("reference_100M.txt","mammoth_reads_10M.txt",  "ground_truth_10M.txt"),
        #("reference_1T.txt",  "mammoth_reads_100M.txt", "ground_truth_100M.txt"),
    ]

    for ref_file, read_file, truth_file in pairs:
        if not (os.path.exists(ref_file) and os.path.exists(read_file) and os.path.exists(truth_file)):
            print(f"> Skipping {ref_file} / {read_file} / {truth_file}: 파일이 존재하지 않음.")
            continue

        print(f"\n=== Processing {ref_file} & {read_file} ===")
        reference = load_reference_to_string(ref_file)
        reads = load_reads_from_file(read_file)
        truth_positions = load_truth_positions(truth_file)

        print("> Building minimizer index ...")
        index = build_minimizer_index(reference, k=K, w=W, max_occ=MAX_OCC)
        print(f"  - Unique minimizers after filtering: {len(index)}")

        print("> Performing mapping & reconstruction ...")
        reconstructed, matched_reads, total_reads = reconstruct_genome_with_reads(
            reference, reads, truth_positions, index,
            k=K, w=W, max_mismatch=MAX_MM, seed_min=SEED_MIN
        )

        read_level_acc = matched_reads / total_reads * 100
        base_level_acc = evaluate_reconstruction(reference, reconstructed) * 100

        print(f"  * Read-level mapping accuracy (exact 위치 매칭 & ≤{MAX_MM} mismatch): "
              f"{matched_reads}/{total_reads} = {read_level_acc:.2f}%")
        print(f"  * Base-level reconstruction accuracy: {base_level_acc:.2f}%")

if __name__ == "__main__":
    run_mapping_and_evaluation()
